In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
112,2024-08-22,Filipinas Copa Dos Governadores,08:30,TNT Tropang Giga,Meralco Bolts,1.69,2.12,201.5,1.87,1.87,-2.5,1.89,2.58,APQOdOuf,0.591716,0.471698,0.534759,0.534759,0.063414,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.159609,0.000000,0.218301,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
113,2024-08-22,México Lnbp,23:00,Dorados,Diablos Rojos,1.80,1.86,168.5,1.82,1.84,-2.5,2.05,2.12,OppkTGQi,0.555556,0.537634,0.549451,0.543478,0.093190,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,164.26,103.95,0.0,0.0,0.0,0.0,0.023184,0.007728,0.023740,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
114,2024-08-22,México Lnbp,23:30,Panteras,Astros,1.72,1.95,167.5,1.83,1.83,-2.5,1.95,2.24,WAmcRfe4,0.581395,0.512821,0.546448,0.546448,0.094216,0.0,0.8,NaN,NaN,178.206,45.18042,0.253529,215.528,58.527092,0.271552,0.0,236.61,169.68,271.80,0.0,0.0,0.0,0.0,0.088629,0.000000,0.097881,0.0,0.0,inf,0.0,0.0,0.0,-0.5,-0.1,-9.5,0.0,0.0,0.0
115,2024-08-22,Mundo Copa Do Mundo Feminina,15:00,Coreia do Sul F,Mali F,1.99,1.79,145.5,1.80,1.94,-1.5,1.94,2.05,GUmJR2tm,0.502513,0.558659,0.555556,0.515464,0.061172,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,151.36,0.0,0.0,0.0,0.0,0.074826,0.052938,0.038988,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
116,2024-08-22,Mundo Copa Do Mundo Feminina,20:30,Nova Zelândia F,Moçambique F,1.69,2.15,137.5,1.89,1.89,-2.5,1.93,2.54,O0zzOKuC,0.591716,0.465116,0.529101,0.529101,0.056832,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,126.08,107.25,0.0,0.0,0.0,0.0,0.169411,0.000000,0.192991,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
117,2024-08-22,Paraguai Lnb,20:30,Felix Perez Cardozo,Olimpia,2.15,1.59,144.5,1.87,1.79,1.5,2.05,1.45,ppbctwIE,0.465116,0.628931,0.534759,0.558659,0.094047,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,130.50,223.30,0.0,0.0,0.0,0.0,0.211754,0.030912,0.242437,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
118,2024-08-22,República Dominicana Lnb,21:00,Metros de Santiago,Reales de la Vega,1.68,2.01,179.5,1.82,1.84,-2.5,1.93,2.46,jqW2q3UN,0.595238,0.497512,0.549451,0.543478,0.092751,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,163.80,0.0,0.0,0.0,0.0,0.126474,0.007728,0.170736,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,08:30,Filipinas Copa Dos Governadores,TNT Tropang Giga,Meralco Bolts,201.5,1.87,1.0000,Over
1,23:00,México Lnbp,Dorados,Diablos Rojos,168.5,1.82,1.0000,Over
2,23:30,México Lnbp,Panteras,Astros,167.5,1.83,1.0000,Over
3,15:00,Mundo Copa Do Mundo Feminina,Coreia do Sul F,Mali F,145.5,1.80,0.9857,Over
4,20:30,Mundo Copa Do Mundo Feminina,Nova Zelândia F,Moçambique F,137.5,1.89,1.0000,Over
5,21:00,República Dominicana Lnb,Metros de Santiago,Reales de la Vega,179.5,1.82,1.0000,Over
